In [ ]:
# Installation de Transformers
! pip install transformers datasets

In [ ]:
import torch
import transformers
torch.__version__, transformers.__version__

In [ ]:
torch.cuda.is_available()

# Tour d'horizon de 🤗 Transformers

Soyez opérationnel avec 🤗 Transformers ! Que vous soyez un développeur ou un utilisateur lambda, cette visite rapide vous aidera à démarrer et vous montrera comment utiliser le `pipeline()` pour l'inférence, charger un modèle pré-entraîné et un préprocesseur avec une [AutoClass](https://huggingface.co/docs/transformers/main/fr/./model_doc/auto), et entraîner rapidement un modèle avec PyTorch ou TensorFlow. Si vous êtes un débutant, nous vous recommandons de consulter nos tutoriels ou notre [cours](https://huggingface.co/course/chapter1/1) suivant pour des explications plus approfondies des concepts présentés ici.

Avant de commencer, assurez-vous que vous avez installé toutes les bibliothèques nécessaires :

```bash
!pip install transformers datasets
```

Vous aurez aussi besoin d'installer votre bibliothèque d'apprentissage profond favorite :

```bash
pip install torch
```
```bash
pip install tensorflow
```

## Pipeline

Le `pipeline()` est le moyen le plus simple d'utiliser un modèle pré-entraîné pour l'inférence. Vous pouvez utiliser le `pipeline()` prêt à l'emploi pour de nombreuses tâches dans différentes modalités. Consultez le tableau ci-dessous pour connaître les tâches prises en charge :

| **Tâche**                     | **Description**                                                                                              | **Modalité**        | **Identifiant du pipeline**                   |
|------------------------------|--------------------------------------------------------------------------------------------------------------|----------------------|-----------------------------------------------|
| Classification de texte      | Attribue une catégorie à une séquence de texte donnée                                                        | Texte                | pipeline(task="sentiment-analysis")           |
| Génération de texte          | Génère du texte à partir d'une consigne donnée                                                               | Texte                | pipeline(task="text-generation")              |
| Reconnaissance de token nommé      | Attribue une catégorie à chaque token dans une séquence (personnes, organisation, localisation, etc.)                            | Texte                | pipeline(task="ner")                          |
| Question réponse             | Extrait une réponse du texte en fonction du contexte et d'une question                                       | Texte                | pipeline(task="question-answering")           |
| Prédiction de token masqué                    | Prédit correctement le token masqué dans une séquence                                                               | Texte                | pipeline(task="fill-mask")                    |
| Génération de résumé                | Génère un résumé d'une séquence de texte donnée ou d'un document                                                         | Texte                | pipeline(task="summarization")                |
| Traduction                  | Traduit du texte d'un langage à un autre                                                                      | Texte                | pipeline(task="translation")                  |
| Classification d'image       | Attribue une catégorie à une image                                                                           | Image                | pipeline(task="image-classification")         |
| Segmentation d'image           | Attribue une catégorie à chaque pixel d'une image (supporte la segmentation sémantique, panoptique et d'instance) | Image                | pipeline(task="image-segmentation")           |
| Détection d'objects             | Prédit les délimitations et catégories d'objects dans une image                                                | Image                | pipeline(task="object-detection")             |
| Classification d'audio       | Attribue une catégorie à un fichier audio                                                                    | Audio                | pipeline(task="audio-classification")         |
| Reconnaissance automatique de la parole | Extrait le discours d'un fichier audio en texte                                                                  | Audio                | pipeline(task="automatic-speech-recognition") |
| Question réponse visuels    | Etant données une image et une question, répond correctement à une question sur l'image                                   | Modalités multiples  | pipeline(task="vqa")                          |

Commencez par créer une instance de `pipeline()` et spécifiez la tâche pour laquelle vous souhaitez l'utiliser. Vous pouvez utiliser le `pipeline()` pour n'importe laquelle des tâches mentionnées dans le tableau précédent. Pour obtenir une liste complète des tâches prises en charge, consultez la documentation de l'[API pipeline](https://huggingface.co/docs/transformers/main/fr/./main_classes/pipelines). Dans ce guide, nous utiliserons le `pipeline()` pour l'analyse des sentiments à titre d'exemple :

In [ ]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")

Le `pipeline()` télécharge et stocke en cache un [modèle pré-entraîné](https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english) et un tokenizer par défaut pour l'analyse des sentiments. Vous pouvez maintenant utiliser le `classifier` sur le texte de votre choix :

In [ ]:
classifier("We are very happy to show you the 🤗 Transformers library.")

Si vous voulez classifier plus qu'un texte, donnez une liste de textes au `pipeline()` pour obtenir une liste de dictionnaires en retour :

In [ ]:
results = classifier(["We are very happy to show you the 🤗 Transformers library.", "We hope you don't hate it."])
for result in results:
    print(f"label: {result['label']}, avec le score de: {round(result['score'], 4)}")

Le `pipeline()` peut aussi itérer sur un jeu de données entier pour n'importe quelle tâche. Prenons par exemple un jeu de données d'avis sur IMDB (voir le 🤗 Datasets [Quick Start](https://huggingface.co/docs/datasets/quickstart#audio) pour plus de détails):

In [ ]:
from datasets import load_dataset

dataset = load_dataset("imdb", split="train")

L'objet retourné se comporte comme une liste de dictionnaire :

In [ ]:
dataset[5]['text']

In [ ]:
dataset[5]['label']

Nous pouvons passer ce dataset comme une liste au pipeline :

In [ ]:
result = classifier(dataset[:6]["text"])
print([d["label"] for d in result])

Pour les ensembles de données plus importants où les entrées sont volumineuses (comme dans les domaines de la parole ou de la vision), utilisez plutôt un générateur au lieu d'une liste pour charger toutes les entrées en mémoire. Pour plus d'informations, consultez la documentation de l'[API pipeline](https://huggingface.co/docs/transformers/main/fr/./main_classes/pipelines).

### Utiliser une autre modèle et tokenizer dans le pipeline

Le `pipeline()` peut être utilisé avec n'importe quel modèle du [Hub](https://huggingface.co/models), ce qui permet d'adapter facilement le `pipeline()` à d'autres cas d'utilisation. Par exemple, si vous souhaitez un modèle capable de traiter du texte français, utilisez les filtres du Hub pour trouver un modèle approprié. Le premier résultat renvoie un [modèle BERT](https://huggingface.co/nlptown/bert-base-multilingual-uncased-sentiment) multilingue finetuné pour l'analyse des sentiments que vous pouvez utiliser pour le texte français :

In [ ]:
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"

Utilisez `AutoModelForSequenceClassification` et `AutoTokenizer` pour charger le modèle pré-entraîné et le tokenizer adapté (plus de détails sur une `AutoClass` dans la section suivante) :

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Utilisez `AutoModelForSequenceClassification` et `AutoTokenizer` pour charger le modèle pré-entraîné et le tokenizer adapté (plus de détails sur une `AutoClass` dans la section suivante) :

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Specifiez le modèle et le tokenizer dans le `pipeline()`, et utilisez le `classifier` sur le texte en français :

In [ ]:
classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
classifier("Nous sommes très heureux de vous présenter la bibliothèque 🤗 Transformers.")

Si vous ne parvenez pas à trouver un modèle adapté à votre cas d'utilisation, vous devrez finetuner un modèle pré-entraîné sur vos données. Jetez un coup d'œil à notre [tutoriel sur le finetuning](https://huggingface.co/docs/transformers/main/fr/./training) pour apprendre comment faire. Enfin, après avoir finetuné votre modèle pré-entraîné, pensez à [partager](https://huggingface.co/docs/transformers/main/fr/./model_sharing) le modèle avec la communauté sur le Hub afin de démocratiser l'apprentissage automatique pour tous ! 🤗

## AutoClass

Les classes `AutoModelForSequenceClassification` et `AutoTokenizer` fonctionnent ensemble pour créer un `pipeline()` comme celui que vous avez utilisé ci-dessus. Une [AutoClass](https://huggingface.co/docs/transformers/main/fr/./model_doc/auto) est un raccourci qui récupère automatiquement l'architecture d'un modèle pré-entraîné à partir de son nom ou de son emplacement. Il vous suffit de sélectionner l'`AutoClass` appropriée à votre tâche et la classe de prétraitement qui lui est associée. 

Reprenons l'exemple de la section précédente et voyons comment vous pouvez utiliser l'`AutoClass` pour reproduire les résultats du `pipeline()`.

### AutoTokenizer

Un tokenizer est chargé de prétraiter le texte pour en faire un tableau de chiffres qui servira d'entrée à un modèle. De nombreuses règles régissent le processus de tokenisation, notamment la manière de diviser un mot et le niveau auquel les mots doivent être divisés (pour en savoir plus sur la tokenisation, consultez le [résumé](https://huggingface.co/docs/transformers/main/fr/./tokenizer_summary)). La chose la plus importante à retenir est que vous devez instancier un tokenizer avec le même nom de modèle pour vous assurer que vous utilisez les mêmes règles de tokenisation que celles avec lesquelles un modèle a été pré-entraîné.

Chargez un tokenizer avec `AutoTokenizer` :

In [ ]:
from transformers import AutoTokenizer

model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)

Passez votre texte au tokenizer :

In [ ]:
encoding = tokenizer("We are very happy to show you the 🤗 Transformers library.")
print(encoding)

Le tokenizer retourne un dictionnaire contenant :

* [input_ids](https://huggingface.co/docs/transformers/main/fr/./glossary#input-ids): la représentation numérique des tokens.
* [attention_mask](https://huggingface.co/docs/transformers/main/fr/.glossary#attention-mask): indique quels tokens doivent faire l'objet d'une attention particulière (plus particulièrement les tokens de remplissage).

Un tokenizer peut également accepter une liste de textes, et remplir et tronquer le texte pour retourner un échantillon de longueur uniforme :

In [ ]:
pt_batch = tokenizer(
    ["We are very happy to show you the 🤗 Transformers library.", "We hope you don't hate it."],
    padding=True,
    truncation=True,
    max_length=512,
    return_tensors="pt",
)

In [ ]:
from pprint import pprint
display(pt_batch)

<Tip>

Consultez le tutoriel [prétraitement](https://huggingface.co/docs/transformers/main/fr/./preprocessing) pour plus de détails sur la tokenisation, et sur la manière d'utiliser un `AutoImageProcessor`, un `AutoFeatureExtractor` et un `AutoProcessor` pour prétraiter les images, l'audio et les contenus multimodaux.

</Tip>

### AutoModel

🤗 Transformers fournit un moyen simple et unifié de charger des instances pré-entraînées. Cela signifie que vous pouvez charger un `AutoModel` comme vous chargeriez un `AutoTokenizer`. La seule différence est de sélectionner l'`AutoModel` approprié pour la tâche. Pour une classification de texte (ou de séquence de textes), vous devez charger `AutoModelForSequenceClassification` :

In [ ]:
from transformers import AutoModelForSequenceClassification

model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
pt_model = AutoModelForSequenceClassification.from_pretrained(model_name)

<Tip>

Voir le [résumé de la tâche](https://huggingface.co/docs/transformers/main/fr/./task_summary) pour vérifier si elle est prise en charge par une classe `AutoModel`.

</Tip>

Maintenant, passez votre échantillon d'entrées prétraitées directement au modèle. Il vous suffit de décompresser le dictionnaire en ajoutant `**` :

In [ ]:
pt_outputs = pt_model(**pt_batch)

Le modèle produit les activations finales dans l'attribut `logits`. Appliquez la fonction softmax aux `logits` pour récupérer les probabilités :

In [ ]:
from torch import nn

pt_predictions = nn.functional.softmax(pt_outputs.logits, dim=-1)
print(pt_predictions)

<Tip>

Tous les modèles 🤗 Transformers (PyTorch ou TensorFlow) produisent les tensors *avant* la fonction d'activation finale (comme softmax) car la fonction d'activation finale est souvent fusionnée avec le calcul de la perte. Les structures produites par le modèle sont des classes de données spéciales, de sorte que leurs attributs sont autocomplétés dans un environnement de développement. Les structures produites par le modèle se comportent comme un tuple ou un dictionnaire (vous pouvez les indexer avec un entier, une tranche ou une chaîne), auquel cas les attributs qui sont None sont ignorés.

</Tip>

### Sauvegarder un modèle

Si vous finetunez ou entrainez un modèle, vous pouvez le sauvegarder avec son tokenizer en utilisant `PreTrainedModel.save_pretrained()` :

In [ ]:
pt_save_directory = "./pt_save_pretrained"
tokenizer.save_pretrained(pt_save_directory)
pt_model.save_pretrained(pt_save_directory)

Lorsque vous voulez réutiliser le modèle, rechargez-le avec `PreTrainedModel.from_pretrained()` :

In [ ]:
pt_model = AutoModelForSequenceClassification.from_pretrained("./pt_save_pretrained")